In [1]:
#r "nuget:Microsoft.Spark"

Installed Packages Microsoft.Spark, 2.1.1

Loading extensions from `C:\Users\marco\.nuget\packages\microsoft.data.analysis\0.18.0\interactive-extensions\dotnet\Microsoft.Data.Analysis.Interactive.dll`

In [2]:
using System;
using System.IO;
using Microsoft.Spark;
using Microsoft.Spark.Sql;
using static Microsoft.Spark.Sql.Functions;

In [3]:
long rows = 1_000_000_000;

In [4]:
// Create a Spark session
SparkSession spark = SparkSession
    .Builder()
    .AppName("montecarlo")
    .GetOrCreate();


[2023-02-08T23:42:06.4213273Z] [DESKTOP-KAD9IHR] [Info] [ConfigurationService] 'DOTNETBACKEND_PORT' environment variable is not set.
[2023-02-08T23:42:06.4235891Z] [DESKTOP-KAD9IHR] [Info] [ConfigurationService] Using port 5567 for connection.
[2023-02-08T23:42:06.4242871Z] [DESKTOP-KAD9IHR] [Info] [JvmBridge] JvMBridge port is 5567
[2023-02-08T23:42:06.4253445Z] [DESKTOP-KAD9IHR] [Info] [JvmBridge] The number of JVM backend thread is set to 10. The max number of concurrent sockets in JvmBridge is set to 7.


In [5]:
DataFrame dataFrame = spark
    .Range(0, rows)
    .Select(Rand().Multiply(2).Minus(1).As("x"), Rand().Multiply(2).Minus(1).As("y"))
    .Select(
        When(
            Col("x").Multiply(Col("x")).Plus(Col("y").Multiply(Col("y"))).Leq(1), 1
        )
        .Otherwise(0)
        .As("InCircle"))
    .Agg(Sum(Col("InCircle")).As("t"))
    .Select(Col("t").Multiply(4).Divide(rows).As("P1"))
    ;

In [6]:
var start = DateTime.Now;

dataFrame.Show();

var stop = DateTime.Now;
Console.WriteLine($"Total Seconds: {(stop-start).TotalSeconds}");

+-----------+
|         P1|
+-----------+
|3.141555136|
+-----------+

Total Seconds: 16,0582492


In [7]:
// Stop Spark session
spark.Stop();